In [1]:
# https://mne.tools/stable/auto_tutorials/inverse/35_dipole_orientations.html

import mne
import numpy as np
from mne.datasets import sample
from mne.minimum_norm import make_inverse_operator, apply_inverse

data_path = sample.data_path()
meg_path = data_path / "MEG" / "sample"
evokeds = mne.read_evokeds(meg_path / "sample_audvis-ave.fif")
left_auditory = evokeds[0].apply_baseline()
# fwd = mne.read_forward_solution(meg_path / "sample_audvis-meg-eeg-oct-6-fwd.fif")
fwd = mne.read_forward_solution('sample_fsaverage_fwd.fif')
mne.convert_forward_solution(fwd, surf_ori=True, copy=False)
noise_cov = mne.read_cov(meg_path / "sample_audvis-cov.fif")
# subject = 'sample'
subject = "fsaverage"
subjects_dir = data_path / "subjects"
# trans_fname = meg_path / "sample_audvis_raw-trans.fif"
trans_fname = 'sample_fsaverage_trans.fif'


Reading /home/zhibinz2/mne_data/MNE-sample-data/MEG/sample/sample_audvis-ave.fif ...
    Read a total of 4 projection items:
        PCA-v1 (1 x 102) active
        PCA-v2 (1 x 102) active
        PCA-v3 (1 x 102) active
        Average EEG reference (1 x 60) active
    Found the data of interest:
        t =    -199.80 ...     499.49 ms (Left Auditory)
        0 CTF compensation matrices available
        nave = 55 - aspect type = 100
Projections have already been applied. Setting proj attribute to True.
No baseline correction applied
    Read a total of 4 projection items:
        PCA-v1 (1 x 102) active
        PCA-v2 (1 x 102) active
        PCA-v3 (1 x 102) active
        Average EEG reference (1 x 60) active
    Found the data of interest:
        t =    -199.80 ...     499.49 ms (Right Auditory)
        0 CTF compensation matrices available
        nave = 61 - aspect type = 100
Projections have already been applied. Setting proj attribute to True.
No baseline correction applied


In [2]:
lh = fwd["src"][0]  # Visualize the left hemisphere
verts = lh["rr"]  # The vertices of the source space
tris = lh["tris"]  # Groups of three vertices that form triangles
dip_pos = lh["rr"][lh["vertno"]]  # The position of the dipoles
dip_ori = lh["nn"][lh["vertno"]]
dip_len = len(dip_pos)
dip_times = [0]
white = (1.0, 1.0, 1.0)  # RGB values for a white color

actual_amp = np.ones(dip_len)  # misc amp to create Dipole instance
actual_gof = np.ones(dip_len)  # misc GOF to create Dipole instance
dipoles = mne.Dipole(dip_times, dip_pos, actual_amp, dip_ori, actual_gof)
trans = mne.read_trans(trans_fname)

fig = mne.viz.create_3d_figure(size=(600, 400), bgcolor=white)
coord_frame = "mri"

# Plot the cortex
mne.viz.plot_alignment(
    subject=subject,
    subjects_dir=subjects_dir,
    trans=trans,
    surfaces="white",
    coord_frame=coord_frame,
    fig=fig,
)

# Mark the position of the dipoles with small red dots
mne.viz.plot_dipole_locations(
    dipoles=dipoles,
    trans=trans,
    mode="sphere",
    subject=subject,
    subjects_dir=subjects_dir,
    coord_frame=coord_frame,
    scale=7e-4,
    fig=fig,
)

mne.viz.set_3d_view(figure=fig, azimuth=180, distance=0.25)

Using pyvistaqt 3d backend.



qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in ""


In [3]:
fig = mne.viz.create_3d_figure(size=(600, 400))

# Plot the cortex
mne.viz.plot_alignment(
    subject=subject,
    subjects_dir=subjects_dir,
    trans=trans,
    surfaces="white",
    coord_frame="head",
    fig=fig,
)

# Show the dipoles as arrows pointing along the surface normal
mne.viz.plot_dipole_locations(
    dipoles=dipoles,
    trans=trans,
    mode="arrow",
    subject=subject,
    subjects_dir=subjects_dir,
    coord_frame="head",
    scale=7e-4,
    fig=fig,
)

mne.viz.set_3d_view(figure=fig, azimuth=180, distance=0.1)

In [4]:
# Fixed dipole orientations
# Compute the source estimate for the left auditory condition in the sample
# dataset.
inv = make_inverse_operator(left_auditory.info, fwd, noise_cov, fixed=True)
stc = apply_inverse(left_auditory, inv, pick_ori=None)

# Visualize it at the moment of peak activity.
_, time_max = stc.get_peak(hemi="lh")
brain_fixed = stc.plot(
    surface="white",
    subjects_dir=subjects_dir,
    initial_time=time_max,
    time_unit="s",
    size=(600, 400),
)
mne.viz.set_3d_view(figure=brain_fixed, focalpoint=(0.0, 0.0, 50))

Computing inverse operator with 305 channels.
    305 out of 306 channels remain after picking
Selected 305 channels
Creating the depth weighting matrix...
    203 planar channels
    limit = 19844/20484 = 10.001610
    scale = 2.28722e-08 exp = 0.8
    Picked elements from a free-orientation depth-weighting prior into the fixed-orientation one
    No patch info available. The standard source space normals will be employed in the rotation to the local surface coordinates....
    Changing to fixed-orientation forward solution with surface-based source orientations...
    [done]
Whitening the forward solution.
    Created an SSP operator (subspace dimension = 3)
Computing rank from covariance with rank=None
    Using tolerance 3.3e-13 (2.2e-16 eps * 305 dim * 4.8  max singular value)
    Estimated rank (mag + grad): 302
    MEG: rank 302 computed from 305 data channels with 3 projectors
    Setting small MEG eigenvalues to zero (without PCA)
Creating the source covariance matrix
Adjustin

In [5]:
fig = mne.viz.create_3d_figure(size=(600, 400))

# Plot the cortex
mne.viz.plot_alignment(
    subject=subject,
    subjects_dir=subjects_dir,
    trans=trans,
    surfaces="white",
    coord_frame="head",
    fig=fig,
)

# Show the three dipoles defined at each location in the source space
mne.viz.plot_alignment(
    subject=subject,
    subjects_dir=subjects_dir,
    trans=trans,
    fwd=fwd,
    surfaces="white",
    coord_frame="head",
    fig=fig,
)

mne.viz.set_3d_view(figure=fig, azimuth=180, distance=0.1)

In [6]:
# Loose dipole orientations
# Make an inverse operator with loose dipole orientations
inv = make_inverse_operator(left_auditory.info, fwd, noise_cov, fixed=False, loose=1.0)

# Compute the source estimate, indicate that we want a vector solution
stc = apply_inverse(left_auditory, inv, pick_ori="vector")

# Visualize it at the moment of peak activity.
_, time_max = stc.magnitude().get_peak(hemi="lh")
brain_mag = stc.plot(
    subjects_dir=subjects_dir,
    initial_time=time_max,
    time_unit="s",
    size=(600, 400),
    overlay_alpha=0,
)
mne.viz.set_3d_view(figure=brain_mag, focalpoint=(0.0, 0.0, 50))

Computing inverse operator with 305 channels.
    305 out of 306 channels remain after picking
Selected 305 channels
Creating the depth weighting matrix...
    203 planar channels
    limit = 19844/20484 = 10.001610
    scale = 2.28722e-08 exp = 0.8
Whitening the forward solution.
    Created an SSP operator (subspace dimension = 3)
Computing rank from covariance with rank=None
    Using tolerance 3.3e-13 (2.2e-16 eps * 305 dim * 4.8  max singular value)
    Estimated rank (mag + grad): 302
    MEG: rank 302 computed from 305 data channels with 3 projectors
    Setting small MEG eigenvalues to zero (without PCA)
Creating the source covariance matrix
Adjusting source covariance matrix.
Computing SVD of whitened and weighted lead field matrix.
    largest singular value = 5.94151
    scaling factor to adjust the trace = 6.1399e+19 (nchan = 305 nzero = 3)
Preparing the inverse operator for use...
    Scaled noise and source covariance from nave = 1 to nave = 55
    Created the regularized

In [7]:
# Limiting orientations, but not fixing them

# Set loose to 0.2, the default value
inv = make_inverse_operator(left_auditory.info, fwd, noise_cov, fixed=False, loose=0.2)
stc = apply_inverse(left_auditory, inv, pick_ori="vector")

# Visualize it at the moment of peak activity.
_, time_max = stc.magnitude().get_peak(hemi="lh")
brain_loose = stc.plot(
    subjects_dir=subjects_dir,
    initial_time=time_max,
    time_unit="s",
    size=(600, 400),
    overlay_alpha=0,
)
mne.viz.set_3d_view(figure=brain_loose, focalpoint=(0.0, 0.0, 50))

Computing inverse operator with 305 channels.
    305 out of 306 channels remain after picking
Selected 305 channels
Creating the depth weighting matrix...
    203 planar channels
    limit = 19844/20484 = 10.001610
    scale = 2.28722e-08 exp = 0.8
Applying loose dipole orientations to surface source spaces: 0.2
Whitening the forward solution.
    Created an SSP operator (subspace dimension = 3)
Computing rank from covariance with rank=None
    Using tolerance 3.3e-13 (2.2e-16 eps * 305 dim * 4.8  max singular value)
    Estimated rank (mag + grad): 302
    MEG: rank 302 computed from 305 data channels with 3 projectors
    Setting small MEG eigenvalues to zero (without PCA)
Creating the source covariance matrix
Adjusting source covariance matrix.
Computing SVD of whitened and weighted lead field matrix.
    largest singular value = 5.80116
    scaling factor to adjust the trace = 2.76604e+19 (nchan = 305 nzero = 3)
Preparing the inverse operator for use...
    Scaled noise and source

In [8]:
# Only retain vector magnitudes
stc = apply_inverse(left_auditory, inv, pick_ori=None)

# Visualize it at the moment of peak activity
_, time_max = stc.get_peak(hemi="lh")
brain = stc.plot(
    surface="white",
    subjects_dir=subjects_dir,
    initial_time=time_max,
    time_unit="s",
    size=(600, 400),
)
mne.viz.set_3d_view(figure=brain, focalpoint=(0.0, 0.0, 50))

Preparing the inverse operator for use...
    Scaled noise and source covariance from nave = 1 to nave = 55
    Created the regularized inverter
    Created an SSP operator (subspace dimension = 3)
    Created the whitener using a noise covariance matrix with rank 302 (3 small eigenvalues omitted)
    Computing noise-normalization factors (dSPM)...
[done]
Applying inverse operator to "Left Auditory"...
    Picked 305 channels from the data
    Computing inverse...
    Eigenleads need to be weighted ...
    Computing residual...
    Explained  57.7% variance
    Combining the current components...
    dSPM...
[done]
Using control points [ 3.90747883  4.50266095 17.93222459]
